In [1]:
# импортируем необходимые библиотеки, функции и классы
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [2]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/StateFarm_missing.csv', sep=';')
data.head(5)

,Customer Lifetime Value,Coverage,Education,EmploymentStatus,Gender,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Response
0,2763.519279,Basic,Bachelor,Employed,F,56274.0,NaN,32.0,5.0,NaN,1.0,No
1,NaN,NaN,Bachelor,Unemployed,F,0.0,NaN,13.0,42.0,NaN,NaN,No
2,NaN,NaN,NaN,Employed,F,48767.0,108.0,NaN,38.0,0.0,NaN,No
3,7645.861827,Basic,Bachelor,NaN,NaN,0.0,106.0,18.0,NaN,NaN,7.0,No
4,2813.692575,Basic,Bachelor,NaN,M,43836.0,73.0,12.0,NaN,NaN,1.0,No


In [3]:
# разбиваем данные на обучающие и тестовые: получаем обучающий
# массив признаков, тестовый массив признаков, обучающий массив
# меток, тестовый массив меток
X_train, X_test, y_train, y_test = train_test_split(data.drop('Response', axis=1), 
                                                    data['Response'], 
                                                    test_size=0.3,
                                                    stratify=data['Response'],
                                                    random_state=42)

In [4]:
# создаем списки количественных 
# и категориальных столбцов
cat_columns = X_train.dtypes[X_train.dtypes == 'object'].index
num_columns = X_train.dtypes[X_train.dtypes != 'object'].index

In [5]:
# создаем конвейер для количественных переменных
num_pipe = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler())
])

# создаем конвейер для категориальных переменных
cat_pipe = Pipeline([
    ('imputer', SimpleImputer()),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [6]:
# создаем список трехэлементных кортежей, в котором
# первый элемент кортежа - название конвейера с
# преобразованиями для определенного типа признаков
transformers = [('num', num_pipe, num_columns),
                ('cat', cat_pipe, cat_columns)]

In [7]:
# передаем список трансформеров в ColumnTransformer
transformer = ColumnTransformer(transformers=transformers)

In [8]:
# задаем итоговый конвейер
ml_pipe = Pipeline([('transform', transformer), 
                    ('logreg', LogisticRegression(solver='lbfgs', max_iter=200))])

In [9]:
# задаем сетку гиперпараметров
param_grid = {
    'transform__num__imputer__strategy': ['mean', 'median', 'constant'],
    'transform__cat__imputer__strategy': ['most_frequent', 'constant'],
    'logreg__C': [.01, .1, .5, 1, 5, 10, 100]
}
# создаем экземпляр класса GridSearchCV, передав конвейер,
# сетку гиперпараметров и указав количество
# блоков перекрестной проверки, отключив запись метрик 
# для обучающих блоков перекрестной проверки в атрибут cv_results_
gs = GridSearchCV(ml_pipe, param_grid, cv=5, return_train_score=False)
# выполняем решетчатый поиск
gs.fit(X_train, y_train)
# смотрим наилучшие значения гиперпараметров
print('Наилучшие значения гиперпараметров: {}'.format(gs.best_params_))
# смотрим наилучшее значение правильности
print('Наилучшее значение правильности: {:.3f}'.format(gs.best_score_))
# смотрим значение правильности
# на тестовой выборке
print('Значение правильности на тестовой выборке: {:.3f}'.format(gs.score(X_test, y_test)))

Наилучшие значения гиперпараметров: {'logreg__C': 0.01, 'transform__cat__imputer__strategy': 'most_frequent', 'transform__num__imputer__strategy': 'mean'}
Наилучшее значение правильности: 0.900
Значение правильности на тестовой выборке: 0.900


In [10]:
# запишем результаты перекрестной 
# проверки в DataFrame
results = pd.DataFrame(gs.cv_results_)
# превращаем в сводную таблицу
table = results.pivot_table(values=['mean_test_score'],    
                            index=['param_logreg__C', 
                                   'param_transform__cat__imputer__strategy',
                                   'param_transform__num__imputer__strategy'])
print(table)

                                                                                                 mean_test_score
param_logreg__C param_transform__cat__imputer__strategy param_transform__num__imputer__strategy                 
0.01            constant                                constant                                        0.899742
                                                        mean                                            0.899742
                                                        median                                          0.899742
                most_frequent                           constant                                        0.899742
                                                        mean                                            0.899742
                                                        median                                          0.899742
0.10            constant                                constant                                

In [11]:
# записываем CSV-файл в объект DataFrame
data2 = pd.read_csv('Data/Response.csv', encoding='cp1251', sep=';')
data2.head(5)

,mortgage,life_ins,cre_card,deb_card,mob_bank,curr_acc,internet,perloan,savings,atm_user,markpl,age,cus_leng,response
0,No,No,No,No,No,No,No,No,No,No,No,18.0,less than 3 years,No
1,Yes,Yes,NaN,NaN,Yes,No,NaN,NaN,NaN,Yes,No,18.0,NaN,Yes
2,Yes,Yes,NaN,Yes,No,No,No,No,No,No,Yes,NaN,from 3 to 7 years,Yes
3,Yes,Yes,Yes,Yes,NaN,Yes,No,No,No,NaN,Yes,18.0,from 3 to 7 years,Yes
4,Yes,Yes,No,Yes,No,No,No,Yes,No,Yes,No,NaN,NaN,No


In [12]:
# создаем обучающий массив признаков, обучающий массив меток,
# тестовый массив признаков, тестовый массив меток
tr, tst, y_tr, y_tst = train_test_split(data2.drop('response', axis=1), 
                                        data2['response'], 
                                        test_size=.3, 
                                        stratify=data2['response'], 
                                        random_state=100)

In [13]:
# создаем списки количественных 
# и категориальных столбцов
categorical_features = tr.dtypes[tr.dtypes == 'object'].index
numeric_features = tr.dtypes[tr.dtypes != 'object'].index

# создаем трансформеры
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))])

# передаем список трансформеров в ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# формируем итоговый конвейер
pipe2 = Pipeline([('preprocessor', preprocessor),
                  ('classifier', LogisticRegression(solver='lbfgs', max_iter=400))])  

In [14]:
# задаем сетку гиперпараметров
param_grid2 = [
    {'classifier': [GradientBoostingClassifier(n_estimators=50, 
                                               random_state=42,
                                               subsample=0.8)],
     'classifier__max_depth': [4, 6, 8],
     'preprocessor__num__scaler': [None]},
    {'classifier': [LogisticRegression(solver='lbfgs', max_iter=400)],
     'classifier__C': [.05, .01], 
     'preprocessor__num__scaler': [None]},
    {'classifier': [LogisticRegression(solver='lbfgs', max_iter=400)],
     'classifier__C': [.05, .01]}]

In [15]:
# создаем экземпляр класса KFold
kf = KFold(n_splits=5, shuffle=True, random_state=123)
# создаем экземпляр класса GridSearchCV, передав конвейер,
# сетку гиперпараметров, оптимизируемую метрику, указав
# стратегию перекрестной проверки, отключив запись метрик 
# для обучающих блоков перекрестной проверки в атрибут cv_results_
gs2 = GridSearchCV(pipe2, param_grid2, scoring='roc_auc', 
                   cv=kf, return_train_score=False)
# выполняем решетчатый поиск
gs2.fit(tr, y_tr)
# смотрим наилучшие значения гиперпараметров
print('Наилучшие значения гиперпараметров: {}'.format(gs2.best_params_))
# смотрим наилучшее значение AUC
print('Наилучшее значение AUC: {:.3f}'.format(gs2.best_score_))
# смотрим значение AUC на тестовой выборке
print('AUC на тестовом наборе: {:.3f}'.format(
    roc_auc_score(y_tst, gs2.predict_proba(tst)[:, 1])))

Наилучшие значения гиперпараметров: {'classifier': GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=4,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=50,
              n_iter_no_change=None, presort='auto', random_state=42,
              subsample=0.8, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False), 'classifier__max_depth': 4, 'preprocessor__num__scaler': None}
Наилучшее значение AUC: 0.910
AUC на тестовом наборе: 0.907
